# Slope Analysis Demo

This notebook demonstrates slope analysis capabilities of TopoConvert.

In [ ]:
import subprocess
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

# Set up paths
data_dir = Path("../data")
output_dir = Path("../data/output")
output_dir.mkdir(exist_ok=True)

## Generate Slope Heatmap

Create a slope analysis heatmap from elevation data.

In [ ]:
# Generate slope heatmap in degrees
result = subprocess.run([
    "topoconvert", "slope-heatmap",
    str(data_dir / "sample.kml"),
    str(output_dir / "slope_degrees.png"),
    "--slope-units", "degrees",
    "--resolution", "1.0",
    "--colormap", "RdYlGn_r"
], capture_output=True, text=True)

print("Output:", result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

## Compare Different Slope Units

Generate slope maps using different units for comparison.

In [ ]:
# Generate slope maps with different units
units = ["degrees", "percent", "ratio"]
colormaps = ["RdYlGn_r", "terrain", "viridis"]

for unit, cmap in zip(units, colormaps):
    output_file = output_dir / f"slope_{unit}.png"
    
    result = subprocess.run([
        "topoconvert", "slope-heatmap",
        str(data_dir / "sample.kml"),
        str(output_file),
        "--slope-units", unit,
        "--resolution", "0.5",
        "--colormap", cmap
    ], capture_output=True, text=True)
    
    print(f"\nGenerated slope map in {unit}")
    if result.stderr:
        print("Errors:", result.stderr)

## Visualize Results

Display the generated slope maps (if matplotlib and PIL are available).

In [ ]:
# Try to display the images
try:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for i, unit in enumerate(units):
        img_path = output_dir / f"slope_{unit}.png"
        if img_path.exists():
            img = Image.open(img_path)
            axes[i].imshow(img)
            axes[i].set_title(f"Slope in {unit}")
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Could not display images: {e}")
    print("\nGenerated files:")
    for unit in units:
        img_path = output_dir / f"slope_{unit}.png"
        if img_path.exists():
            print(f"  - {img_path}")

## Generate Terrain Mesh

Create a triangulated mesh for 3D visualization.

In [ ]:
# Generate Delaunay triangulation
result = subprocess.run([
    "topoconvert", "kml-to-mesh",
    str(data_dir / "sample.kml"),
    str(output_dir / "terrain_mesh.dxf"),
    "--mesh-type", "delaunay"
], capture_output=True, text=True)

print("Delaunay mesh output:", result.stdout)

# Generate concave hull mesh
result = subprocess.run([
    "topoconvert", "kml-to-mesh",
    str(data_dir / "sample.kml"),
    str(output_dir / "terrain_concave.dxf"),
    "--mesh-type", "concave",
    "--alpha", "0.5"
], capture_output=True, text=True)

print("\nConcave hull output:", result.stdout)

## Summary

This demo showed how to:
1. Generate slope analysis heatmaps
2. Compare different slope unit representations
3. Create terrain meshes for 3D visualization

The generated files can be used in GIS software or CAD applications for further analysis.